In [1]:
import os
import glob
import random
import math
import json
from pathlib import Path
import pandas as pd
from sklearn.model_selection import GroupShuffleSplit
import torch
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import torchvision.transforms as T
import timm
import torch.nn as nn
from torch.optim import AdamW
from torch.optim.lr_scheduler import OneCycleLR
from torchmetrics.classification import MulticlassAccuracy
from tqdm.auto import tqdm
import torch.nn.functional as F
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np


/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2225: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2225: UnsupportedFieldAttributeWarning: The 'frozen' attribute with value True was provided to the `Field()` function, which has no effect in the context it was used. 'frozen' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` 

In [2]:
DATA_ROOT = Path("/kaggle/input/behaviours-features-merged/Behaviors_Features_Final")

# Collect all images recursively and derive labels from behavior folder name.
records = []
for behavior_dir in sorted([p for p in DATA_ROOT.iterdir() if p.is_dir()]):
    behavior = behavior_dir.name  # e.g., 'Looking_Forward'
    for id_dir in behavior_dir.glob("*"):
        if not id_dir.is_dir(): 
            continue
        for seq_dir in id_dir.glob("*"):
            if not seq_dir.is_dir():
                continue
            # Group key: person+sequence folder to avoid near-duplicate leakage
            group_key = f"{behavior}/{id_dir.name}/{seq_dir.name}"
            for img_path in seq_dir.rglob("*.png"):
                records.append({
                    "path": str(img_path),
                    "label": behavior,
                    "group": group_key,
                    "person": id_dir.name,
                    "sequence": seq_dir.name,
                })

df = pd.DataFrame(records)
print("Total images:", len(df))
df.head()


Total images: 632620


,path,label,group,person,sequence
0,/kaggle/input/behaviours-features-merged/Behav...,Looking_Forward,Looking_Forward/ID6/Forward44_id6_Act1_rgb,ID6,Forward44_id6_Act1_rgb
1,/kaggle/input/behaviours-features-merged/Behav...,Looking_Forward,Looking_Forward/ID6/Forward44_id6_Act1_rgb,ID6,Forward44_id6_Act1_rgb
2,/kaggle/input/behaviours-features-merged/Behav...,Looking_Forward,Looking_Forward/ID6/Forward44_id6_Act1_rgb,ID6,Forward44_id6_Act1_rgb
3,/kaggle/input/behaviours-features-merged/Behav...,Looking_Forward,Looking_Forward/ID6/Forward44_id6_Act1_rgb,ID6,Forward44_id6_Act1_rgb
4,/kaggle/input/behaviours-features-merged/Behav...,Looking_Forward,Looking_Forward/ID6/Forward44_id6_Act1_rgb,ID6,Forward44_id6_Act1_rgb


In [3]:
# Map class names to indices; keep a clean label list for the model head.
class_names = sorted(df["label"].unique())
class2idx = {c:i for i,c in enumerate(class_names)}
df["y"] = df["label"].map(class2idx)

# First split: train+val vs test by groups (sequence level).
gss = GroupShuffleSplit(n_splits=1, test_size=0.15, random_state=42)
trainval_idx, test_idx = next(gss.split(df, groups=df["group"]))
df_trainval, df_test = df.iloc[trainval_idx].reset_index(drop=True), df.iloc[test_idx].reset_index(drop=True)

# Second split: train vs val (still grouped to prevent leakage).
gss2 = GroupShuffleSplit(n_splits=1, test_size=0.15, random_state=123)
tr_idx, va_idx = next(gss2.split(df_trainval, groups=df_trainval["group"]))
df_train, df_val = df_trainval.iloc[tr_idx].reset_index(drop=True), df_trainval.iloc[va_idx].reset_index(drop=True)

print(len(df_train), len(df_val), len(df_test))
class_names


464108 77042 91470


['Looking_Forward',
 'Raising_Hand',
 'Reading',
 'Sleeping',
 'Standing',
 'Turning_Around',
 'Writing']

In [4]:
IMG_SIZE = 224  # You can try 256 or 384 later.

# Training-time augmentations for robustness.
train_tfms = T.Compose([
    T.RandomResizedCrop(IMG_SIZE, scale=(0.7, 1.0)),
    T.RandomHorizontalFlip(p=0.5),
    T.RandomRotation(degrees=10),
    T.ToTensor(),
    T.Normalize(mean=(0.485,0.456,0.406), std=(0.229,0.224,0.225)),  # ImageNet stats
])

# Validation/Test transforms must be deterministic.
valid_tfms = T.Compose([
    T.Resize(int(IMG_SIZE*1.14)),
    T.CenterCrop(IMG_SIZE),
    T.ToTensor(),
    T.Normalize(mean=(0.485,0.456,0.406), std=(0.229,0.224,0.225)),
])

class BehaviorDataset(Dataset):
    # This dataset reads image paths and returns (tensor, label)
    def __init__(self, df, transforms):
        self.paths = df["path"].tolist()
        self.labels = df["y"].astype(int).tolist()
        self.transforms = transforms

    def __len__(self):
        return len(self.paths)

    def __getitem__(self, idx):
        p = self.paths[idx]
        y = self.labels[idx]
        img = Image.open(p).convert("RGB")
        img = self.transforms(img)
        return img, y

train_ds = BehaviorDataset(df_train, train_tfms)
val_ds   = BehaviorDataset(df_val, valid_tfms)
test_ds  = BehaviorDataset(df_test, valid_tfms)

BATCH_SIZE = 64
NUM_WORKERS = 2  # Kaggle often limits >2; adjust if needed.

train_dl = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True,  num_workers=NUM_WORKERS, pin_memory=True)
val_dl   = DataLoader(val_ds,   batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS, pin_memory=True)
test_dl  = DataLoader(test_ds,  batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS, pin_memory=True)


In [5]:
SEED = 42

torch.manual_seed(SEED)
np.random.seed(SEED)
random.seed(SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

N_CLASSES = len(class_names)
MODEL_NAME = "convnext_small_in22ft1k"

# Create a timm model with a classification head sized to our classes.
model = timm.create_model(
    MODEL_NAME,
    pretrained=True,
    num_classes=N_CLASSES,
    drop_path_rate=0.2,
)
model.to(device)
use_amp = device.type == "cuda"

# Loss, optimizer, scheduler, metrics.
criterion = nn.CrossEntropyLoss(label_smoothing=0.1)
optimizer = AdamW(model.parameters(), lr=5e-4, weight_decay=1e-4)
EPOCHS = 10  # Adjust upward for a stronger baseline
steps_per_epoch = len(train_dl)
scheduler = OneCycleLR(optimizer, max_lr=2e-3, epochs=EPOCHS, steps_per_epoch=steps_per_epoch)
metric_acc = MulticlassAccuracy(num_classes=N_CLASSES).to(device)
scaler = torch.cuda.amp.GradScaler(enabled=use_amp)

def run_one_epoch(dataloader, train=True):
    # This function runs one epoch for either training or validation.
    model.train(train)
    total_loss = 0.0
    metric_acc.reset()
    pbar = tqdm(dataloader, leave=False)
    for x, y in pbar:
        x = x.to(device, non_blocking=True)
        y = y.to(device, non_blocking=True)

        with torch.set_grad_enabled(train):
            with torch.cuda.amp.autocast(enabled=use_amp):
                logits = model(x)
                loss = criterion(logits, y)

        if train:
            optimizer.zero_grad()
            scaler.scale(loss).backward()
            scaler.unscale_(optimizer)
            nn.utils.clip_grad_norm_(model.parameters(), max_norm=2.0)
            scaler.step(optimizer)
            scaler.update()
            scheduler.step()
            current_lr = scheduler.get_last_lr()[0]
            pbar.set_postfix(loss=loss.item(), lr=f"{current_lr:.2e}")
        else:
            pbar.set_postfix(loss=loss.item())

        total_loss += loss.item() * x.size(0)
        preds = logits.argmax(dim=1)
        metric_acc.update(preds, y)

    avg_loss = total_loss / len(dataloader.dataset)
    avg_acc = metric_acc.compute().item()
    return avg_loss, avg_acc

best_val = 0.0
for epoch in range(1, EPOCHS+1):
    tr_loss, tr_acc = run_one_epoch(train_dl, train=True)
    va_loss, va_acc = run_one_epoch(val_dl,   train=False)
    print(f"Epoch {epoch:02d} | train loss {tr_loss:.4f} acc {tr_acc:.4f} | val loss {va_loss:.4f} acc {va_acc:.4f}")
    if va_acc > best_val:
        best_val = va_acc
        torch.save({
            "model_name": MODEL_NAME,
            "state_dict": model.state_dict(),
            "class_names": class_names
        }, "/kaggle/working/best_model.pth")
        print("Saved new best model.")


/usr/local/lib/python3.11/dist-packages/timm/models/_factory.py:138: UserWarning: Mapping deprecated model name convnext_small_in22ft1k to current convnext_small.fb_in22k_ft_in1k.
  model = create_fn(


model.safetensors:   0%|          | 0.00/201M [00:00<?, ?B/s]

/tmp/ipykernel_19/1084503723.py:31: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler(enabled=use_amp)


  0%|          | 0/7252 [00:00<?, ?it/s]

/tmp/ipykernel_19/1084503723.py:44: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=use_amp):


...

Epoch 09 | train loss nan acc 0.1429 | val loss nan acc 0.1429


  0%|          | 0/7252 [00:00<?, ?it/s]

Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7a94e1c071a0>
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py", line 1618, in __del__
    self._shutdown_workers()
  File "/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py", line 1601, in _shutdown_workers
    if w.is_alive():
       ^^^^^^^^^^^^
  File "/usr/lib/python3.11/multiprocessing/process.py", line 160, in is_alive
    assert self._parent_pid == os.getpid(), 'can only test a child process'
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
AssertionError: can only test a child process
Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7a94e1c071a0>
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py", line 1618, in __del__
    self._shutdown_workers()
  File "/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py", line 16

  0%|          | 0/1204 [00:00<?, ?it/s]

Epoch 10 | train loss nan acc 0.1429 | val loss nan acc 0.1429


In [6]:

# # Path to your saved version’s files
# MODEL_DIR = "/kaggle/input/classroom-behavior-model"  # ← change to your actual notebook input path

# # Load class names
# with open(f"{MODEL_DIR}/label_map.json", "r") as f:
#     class_names = json.load(f)

# # Recreate the model (same architecture used before)
# MODEL_NAME = "efficientnet_b0"
# N_CLASSES = len(class_names)
# model = timm.create_model(MODEL_NAME, pretrained=False, num_classes=N_CLASSES)

# # Load saved weights
# ckpt = torch.load(f"{MODEL_DIR}/best_model.pth", map_location="cuda")
# model.load_state_dict(ckpt["state_dict"])
# model.eval().to("cuda")

# print("✅ Model reloaded and ready for inference!")


In [7]:
# # Load best weights just in case.
# ckpt = torch.load("/kaggle/working/best_model.pth", map_location=device)
# model.load_state_dict(ckpt["state_dict"])
# model.eval()

# all_preds, all_targs = [], []
# with torch.no_grad():
#     for x, y in tqdm(test_dl):
#         x = x.to(device)
#         logits = model(x)
#         preds = logits.argmax(1).cpu().numpy()
#         all_preds.append(preds)
#         all_targs.append(y.numpy())

# y_pred = np.concatenate(all_preds)
# y_true = np.concatenate(all_targs)

# print(classification_report(y_true, y_pred, target_names=class_names))
# print(confusion_matrix(y_true, y_pred))


In [8]:

# # Reuse valid_tfms for deterministic preprocessing.
# def predict_image(path):
#     # This function predicts the behavior class for a single image path.
#     img = Image.open(path).convert("RGB")
#     x = valid_tfms(img).unsqueeze(0).to(device)
#     with torch.no_grad():
#         logits = model(x)
#         probs = torch.softmax(logits, dim=1).squeeze(0).cpu().numpy()
#     top = probs.argmax()
#     return class_names[top], float(probs[top]), {c: float(p) for c,p in zip(class_names, probs)}

# # Example:
# # predict_image(df_test.iloc[0]["path"])


In [9]:
# # Save class names for future inference.
# with open("/kaggle/working/label_map.json", "w") as f:
#     json.dump(class_names, f, indent=2)

# # The model weights are already at /kaggle/working/best_model.pth


In [10]:
# # Check how many 'group' values overlap between train and test
# set_train_groups = set(df_train["group"])
# set_test_groups = set(df_test["group"])

# len(set_train_groups & set_test_groups)
